In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
##load the ann train model,scaler pickle, onehot
model  = load_model('model.h5')

##load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [30]:
input_data = {
    'CreditScore':700,
    'Geography':'France',
    'Gender':'Male',
    'Age':35,
    'Tenure':1,
    'Balance':6000,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':100000
}

In [31]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Arif\Project\annclassification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [32]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,Male,35,1,6000,1,1,1,100000


In [33]:
##Encode catrogrical  variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,1,35,1,6000,1,1,1,100000


In [34]:
##concatination one hot encoded
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,35,1,6000,1,1,1,100000,1.0,0.0,0.0


In [35]:
##scaling
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 0.50178794,  0.91324755, -0.37056859, -1.38944225, -1.12240462,
        -0.91668767,  0.64920267,  0.97481699, -0.00749869,  1.00150113,
        -0.57946723, -0.57638802]])

In [36]:
##predcit churn
predication = model.predict(input_scaled)
predication

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


array([[0.1390141]], dtype=float32)

In [37]:
predication_proba = predication[0][0]
predication_proba

np.float32(0.1390141)

In [38]:
if predication_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
